In [174]:
import segyio
import matplotlib.patches as patches
import pandas as pd
import re
import plotly.graph_objects as go
import os
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon


In [175]:
def read_seismic_cube(file_path):
  """
  :param file_path: Путь к файлу
  :return: возвращает куб в виде списков по Inlines, Xlines и Sampels (координаты x, y, z)
  а также словарь сейсмотрасс, где ключ это координата пересечения Inlines и Xlines, а значение это список значений сейсмотрасс
  """
  with segyio.open(file_path, 'r') as segyfile:
      x = list(segyfile.ilines)
      y = list(segyfile.xlines)
      z_0 = list(segyfile.samples)
      z = list()
      for i in range(len(z_0)):
          z.append(int(z_0[i]))
      cube = segyio.tools.cube(segyfile)
      #traces = {f"{x[i]}_{y[j]}": cube[i][j][:] for i in range(len(x)) for j in range(len(y))}
  return x, y, z

In [176]:
file_path = 'C:/HV/Seismic/datas/Cube_TWT.segy'
inlines, xlines, samples = read_seismic_cube(file_path)

In [177]:
cube = []
for i in range(len(xlines)):
    for j in range(len(inlines)):
        cube.append([xlines[i], inlines[j]])

In [178]:
print(cube[0:10])

[[5001, 1001], [5001, 1002], [5001, 1003], [5001, 1004], [5001, 1005], [5001, 1006], [5001, 1007], [5001, 1008], [5001, 1009], [5001, 1010]]


In [179]:
def read_seismic_cube_real(file_path):
    """
  :param file_path: Путь к файлу
  :return: возвращает реальные координаты x, y
  """
    with segyio.open(file_path, 'r') as segyfile:
        x = list(segyfile.ilines)
        y = list(segyfile.xlines)
        z = list(segyfile.samples)
        coord_x = set()
        coord_y = set()
        cube_coord = []
        for i in range(len(x)):
            for j in range(len(y)):
                index = i * len(y) + j
                trace_header = segyfile.header[index]
                cube_coord.append([trace_header[segyio.TraceField.SourceX], trace_header[segyio.TraceField.SourceY]])
                #coord_x.add(trace_header[segyio.TraceField.SourceX])
                #coord_y.add(trace_header[segyio.TraceField.SourceY])
    return cube_coord

In [182]:
file_path = 'C:/HV/Seismic/datas/Cube_TWT.segy'
real_cube = read_seismic_cube_real(file_path)

In [117]:
table_result = [['Проверка по оси ХУ', 'TWT_cube', 'Volume_attribute_1', 'Volume_attribute_2', 'top_surface_TWT', 'bot_surface_TWT', 'Surface_attribute', 'Fault_polygons_TWT', 'top_surface_TVDSS', 'bot_surface_TVDSS', 'lateral_trend', 'model_area_boundary', 'licence_area_boundary'],
                ['TWT_cube', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['Volume_attribute 1', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['Volume_attribute 2', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['top_surface_TWT', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['bot_surface_TWT', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['Surface_attribute', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['Fault_polygons_TWT', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['top_surface_TVDSS', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['bot_surface_TVDSS', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['lateral_trend', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['model_area_boundary', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'], 
                ['licence_area_boundary', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'] 
                ]

In [302]:

def peresechenie(coords_polygon1, coords_polygon2, flag, name_polygon1='Полигон 1', name_polygon2='Полигон 2'):
    print('Сравнение ', name_polygon1, ' и ', name_polygon2)
    # Создаем полигоны
    polygon1 = Polygon(coords_polygon1)
    polygon2 = Polygon(coords_polygon2)
    
    # Находим пересечение
    intersection = polygon1.intersection(polygon2)
    
    # Вычисляем площади
    area_polygon1 = polygon1.area
    area_polygon2 = polygon2.area
    area_intersection = intersection.area
    
    # Выводим общую площадь пересечения
    if intersection.is_empty:
        print("Полигоны не пересекаются.")
    else:
        print("Общая площадь пересечения:", area_intersection)
    
        # Вычисляем процент пересечения
        percent_intersection_polygon1 = (area_intersection / area_polygon1) * 100
        percent_intersection_polygon2 = (area_intersection / area_polygon2) * 100
    
        print(f"Процент пересечения относительно {name_polygon1}: {percent_intersection_polygon1:.2f}%")
        print(f"Процент пересечения относительно {name_polygon2}: {percent_intersection_polygon2:.2f}%")
        
    
    
    if flag == 1:
        # Визуализация
        fig, ax = plt.subplots()

    # Отображаем первый полигон
        x1, y1 = polygon1.exterior.xy
        ax.fill(x1, y1, alpha=0.5, fc='blue', label=name_polygon1)

    # Отображаем второй полигон
        x2, y2 = polygon2.exterior.xy
        ax.fill(x2, y2, alpha=0.5, fc='red', label=name_polygon2)

    # Отображаем пересечение
        if not intersection.is_empty:
            x_int, y_int = intersection.exterior.xy
            ax.fill(x_int, y_int, alpha=0.5, fc='green', label='Пересечение')

    # Настройки графика
        ax.set_title(f'Пересечение: {name_polygon1} и {name_polygon2}')
        ax.set_xlabel('Xlines')
        ax.set_ylabel('Inlines')
        ax.legend()
        ax.set_aspect('equal', adjustable='box')
        plt.grid()
    # Добавление легенды справа от графика
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Положение легенды
        plt.show()
    
    return percent_intersection_polygon1, percent_intersection_polygon2


In [310]:
#Функция сравнения для разломов
def peresechenie_2(coords_polygon1, coords_polygon2_list, flag,  name_polygon1='Полигон 1', name_polygon2='Полигон 2'):
    print('Сравнение ', name_polygon1, ' и ', name_polygon2)
    # Создаем первый полигон
    polygon1 = Polygon(coords_polygon1)
    
    if flag == 1:
    # Визуализация
        fig, ax = plt.subplots()

    # Отображаем первый полигон
        x1, y1 = polygon1.exterior.xy
        ax.fill(x1, y1, alpha=0.5, fc='blue', label=name_polygon1)

    # Переменные для хранения результатов
    total_area_intersection = 0
    total_area_polygon2 = 0

    # Обрабатываем каждый полигон из списка coords_polygon2_list
    for coords_polygon2 in coords_polygon2_list:
        # Создаем второй полигон
        polygon2 = Polygon(coords_polygon2)
        
        # Находим пересечение
        intersection = polygon1.intersection(polygon2)
        
        # Вычисляем площади
        area_polygon2 = polygon2.area
        total_area_polygon2 += area_polygon2
        
        # Выводим общую площадь пересечения
        if intersection.is_empty:
            #print(f"Полигоны {name_polygon1} и {name_polygon2} не пересекаются.")
            total_area_intersection += 0
        else:
            area_intersection = intersection.area
            total_area_intersection += area_intersection
    print(f"Общая площадь пересечения: {total_area_intersection}")
    if total_area_intersection == 0:
        print(f"Полигоны {name_polygon1} и {name_polygon2} не пересекаются.")

    # Вычисляем процент пересечения
    if total_area_intersection > 0:
        percent_intersection_polygon1 = (total_area_intersection / polygon1.area) * 100
        percent_intersection_polygon2 = (total_area_intersection / total_area_polygon2) * 100
        
        print(f"Суммарный процент пересечения относительно {name_polygon1}: {percent_intersection_polygon1:.2f}%")
        print(f"Суммарный процент пересечения относительно {name_polygon2}: {percent_intersection_polygon2:.2f}%")
    
    if flag == 1:
    
    # Повторный проход для отображения полигонов и их пересечений
        for coords_polygon2 in coords_polygon2_list:
            polygon2 = Polygon(coords_polygon2)
            x2, y2 = polygon2.exterior.xy
            ax.fill(x2, y2, alpha=0.5, fc='red')  # Легенда только с названием полигона 2

        # Находим пересечение заново для отображения
            intersection = polygon1.intersection(polygon2)
            if not intersection.is_empty:
                if intersection.geom_type == 'Polygon':
                    x_int, y_int = intersection.exterior.xy
                    ax.fill(x_int, y_int, alpha=0.5, fc='green', label='Пересечение')
                elif intersection.geom_type == 'LineString':
                    x_int, y_int = intersection.xy
                    ax.plot(x_int, y_int, color='green', linewidth=2, label='Пересечение')
                elif intersection.geom_type == 'Point':
                    x_int, y_int = intersection.x, intersection.y
                    ax.plot(x_int, y_int, 'go', markersize=8, label='Пересечение')

    # Настройки графика
        ax.set_title(f'Пересечение: {name_polygon1} и {name_polygon2}')
        ax.set_xlabel('Xlines')
        ax.set_ylabel('Inlines')
        ax.legend([name_polygon1, 'Пересечение', name_polygon2], loc='upper left', bbox_to_anchor=(1, 1))  # Обновленная легенда
        ax.set_aspect('equal', adjustable='box')
        plt.grid()
        plt.tight_layout()  # Подгоняем график, чтобы он не выходил за пределы области
        plt.show()
    
    return percent_intersection_polygon1, percent_intersection_polygon2




In [184]:
def create_convex_hull(coords):
    """
    Создает полигон (выпуклую оболочку) из списка координат.

    :param coords: Список координат в формате [[x1, y1], [x2, y2], ...]
    :return: Объект Polygon, представляющий выпуклую оболочку
    """
    if len(coords) < 3:
        raise ValueError("Для создания полигона необходимо как минимум 3 точки.")

    # Преобразуем список координат в массив NumPy 
    points = np.array(coords)

    # Находим выпуклую оболочку
    hull = ConvexHull(points)

    # Извлекаем координаты вершин выпуклой оболочки 
    hull_points = points[hull.vertices]

    # Создаем полигон
    polygon = Polygon(hull_points)
    return polygon

In [185]:
TWT_cube = create_convex_hull(cube)
TWT_cube_real = create_convex_hull(real_cube)

In [121]:
#ридер для объемных атрибутов
def rider(file_path):
    with segyio.open(file_path, 'r') as segyfile:
        y = [i for i in segyfile.xlines]
        x = [i for i in segyfile.ilines]
        cube_coord = []
        for i in range(len(y)):
            for j in range(len(x)):
                cube_coord.append([y[i], x[j]])
    return cube_coord

In [267]:
Volume_attribute_1 = create_convex_hull(read_seismic_cube_real('C:/HV/Seismic/datas/Volume_attribute1_RMS_TWT.segy'))

In [269]:
Volume_attribute_2 = create_convex_hull(read_seismic_cube_real('C:/HV/Seismic/datas/Volume_attribute2_Amplitude_contrast_TWT.segy'))

In [224]:
# ридер для карты кровли и подошвы
def gorizont_coord(file):
    with open(file, 'r') as f:
        gorizon = []
        for line in f:
            if not line.startswith('#'):  # Пропуск строк, начинающихся с #
                x, y, z, *rest = line.split()  # Разбиение строки по столбцам
                x = int(x.split('.')[0])  # Получение целочисленного значения из первого столбца
                y = int(y.split('.')[0])  # Получение целочисленного значения из второго столбца
                #z = int(z.split('.')[0])  # Получение целочисленного значения из третьего столбца
                gorizon.append([x, y])  # Добавление данных в словарь

    return gorizon

In [341]:
top_surface_TWT = create_convex_hull(gorizont_coord('C:/HV/Seismic/datas/TWT_Top_U1.txt'))
bot_surface_TWT = create_convex_hull(gorizont_coord('C:/HV/Seismic/datas/TWT_Bottom_U1.txt'))

In [342]:
Surface_attribute = create_convex_hull(gorizont_coord('C:/HV/Seismic/datas/Surface_attribute_AmplitudeContrast_HalfEnergy_TWT.txt'))

In [343]:
top_surface_TVDSS = create_convex_hull(gorizont_coord('C:/HV/Seismic/datas/TVDSS_Top_U1.txt'))

In [344]:
bot_surface_TVDSS = create_convex_hull(gorizont_coord('C:/HV/Seismic/datas/TVDSS_Bottom_U1.txt'))

In [345]:
lateral_trend = create_convex_hull(gorizont_coord('C:/HV/Seismic/datas/Effective_thickness_calculated_by_surface_attributes.txt'))

In [346]:
#для model_area_boundary и licence_area_boundary
def read_coordinates(file_path):
    coordinates = []
    
    with open(file_path, 'r') as file:
        for line in file:
            # Убираем пробелы и разделяем строку на элементы
            parts = line.strip().split()
            
            # Проверяем, что строка не является строкой конца файла
            if parts[0] == '999.0' and parts[1] == '999.0' and parts[2] == '999.0':
                break
            
            # Преобразуем первую и вторую колонки в int и добавляем в список
            x = int(float(parts[0]))  # Преобразуем в float, а затем в int
            y = int(float(parts[1]))  # Преобразуем в float, а затем в int
            
            coordinates.append([x, y])
    
    return coordinates


In [347]:
model_area_boundary = create_convex_hull(read_coordinates('C:/HV/Seismic/datas/Model_area_boundary_IRAP_ASCII.txt'))

In [348]:
licence_area_boundary = create_convex_hull(read_coordinates('C:/HV/Seismic/datas/Licence_area_boundary_IRAP_ASCII.txt'))

In [349]:
# ридер для разломов
def process_files(directory):
    files = os.listdir(directory)
    result = {}

    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(directory, file), 'r') as f:
                data = []
                for line in f:
                    fields = line.split()
                    if len(fields) >= 8:
                        x = int(float(fields[3]))
                        y = int(float(fields[4]))
                        z = -int(float(fields[5]))
                        t = int(fields[7])
                        data.append([x, y, z, t])
                result[file] = data
    #return result
    
    
    """
    Сборка координат в один список попарно
    Если точек больше 2, то берется минимальное и максимальное значение по глубинам
    Если точка 1, то она не учитывается в разломах
    """
    faults_final = {}

    for key in result:
        dop_spisok = []
        z_spisok = []
        faults_final[key] = []

        for i in range(len(result[key])):
            if len(dop_spisok) == 0:
                dop_spisok.append(result[key][i])
                z_spisok.append(result[key][i][2])
            elif len(dop_spisok) == 1:
                if dop_spisok[0][3] == result[key][i][3]:
                    dop_spisok.append(result[key][i])
                    z_spisok.append(result[key][i][2])
                else:
                    dop_spisok.clear()
                    z_spisok.clear()
                    dop_spisok.append(result[key][i])
                    z_spisok.append(result[key][i][2])
            elif len(dop_spisok) == 2:
                if dop_spisok[0][3] == result[key][i][3]:
                    dop_spisok.append(result[key][i])
                    z_spisok.append(result[key][i][2])
                else:
                    
                    #faults_final.append(dop_spisok[:])
                    faults_final[key].append(dop_spisok[:])
                    dop_spisok.clear()
                    z_spisok.clear()
                    dop_spisok.append(result[key][i])
                    z_spisok.append(result[key][i][2])
            else:
                if dop_spisok[0][3] == result[key][i][3]:
                    dop_spisok.append(result[key][i])
                    z_spisok.append(result[key][i][2])
                else:
                    faults_final[key].append([dop_spisok[z_spisok.index(min(z_spisok))], dop_spisok[z_spisok.index(max(z_spisok))]])
                    dop_spisok.clear()
                    z_spisok.clear()
                    dop_spisok.append(result[key][i])
                    z_spisok.append(result[key][i][2])  
    return faults_final
    
    

In [350]:
directory = 'C:/HV/Seismic/datas/Faults_TWT_Charisma_fault_sticks_ASCII'
faults = process_files(directory)

In [351]:
faults_coord_top = []
j = 0
for key in faults:
    faults_coord_top.append([])
    for i in range(len(faults[key])):
        faults_coord_top[j].append([faults[key][i][0][0], faults[key][i][0][1]])
    j+=1
Fault_polygons_TWT = []
for i in range(len(faults_coord_top)):
    Fault_polygons_TWT.append(create_convex_hull(faults_coord_top[i]))


In [352]:
dannie = [TWT_cube_real, Volume_attribute_1, Volume_attribute_2, top_surface_TWT, bot_surface_TWT, Surface_attribute, Fault_polygons_TWT, top_surface_TVDSS, bot_surface_TVDSS, lateral_trend, model_area_boundary, licence_area_boundary]
dannie_name = ['TWT_cube_real','Volume_attribute_1','Volume_attribute_2','top_surface_TWT','bot_surface_TWT','Surface_attribute','Fault_polygons_TWT','top_surface_TVDSS','bot_surface_TVDSS','lateral_trend','model_area_boundary','licence_area_boundary']

In [358]:
for k in range(1, 12):
    for j in range(k+1, 13):
        if j==7:
            table_result[k][j], table_result[j][k] = peresechenie_2(dannie[k-1], dannie[j-1], 0, dannie_name[k-1], dannie_name[j-1])
        elif k == 7:
            table_result[k][j], table_result[j][k] = peresechenie_2(dannie[j-1], dannie[k-1], 0, dannie_name[k-1], dannie_name[j-1])
        else:
            table_result[k][j], table_result[j][k] = peresechenie(dannie[k-1], dannie[j-1], 0, dannie_name[k-1], dannie_name[j-1])
            
        

1 2 1
Сравнение  TWT_cube_real  и  Volume_attribute_1
Общая площадь пересечения: 236837086.0
Процент пересечения относительно TWT_cube_real: 100.00%
Процент пересечения относительно Volume_attribute_1: 100.00%
1 3 1
Сравнение  TWT_cube_real  и  Volume_attribute_2
Общая площадь пересечения: 236837086.0
Процент пересечения относительно TWT_cube_real: 100.00%
Процент пересечения относительно Volume_attribute_2: 100.00%
1 4 1
Сравнение  TWT_cube_real  и  top_surface_TWT
Общая площадь пересечения: 198787500.0
Процент пересечения относительно TWT_cube_real: 83.93%
Процент пересечения относительно top_surface_TWT: 100.00%
1 5 1
Сравнение  TWT_cube_real  и  bot_surface_TWT
Общая площадь пересечения: 198787500.0
Процент пересечения относительно TWT_cube_real: 83.93%
Процент пересечения относительно bot_surface_TWT: 100.00%
1 6 1
Сравнение  TWT_cube_real  и  Surface_attribute
Общая площадь пересечения: 198787500.0
Процент пересечения относительно TWT_cube_real: 83.93%
Процент пересечения относит

In [287]:
#Вывод таблицы 
data = table_result

# Определяем ширину столбцов
column_widths = [max(len(str(item)) for item in column) for column in zip(*table_result)]

for row in table_result:
    print(" | ".join(f"{str(item):<{column_widths[i]}}" for i, item in enumerate(row)))


Проверка по оси ХУ    | TWT_cube | Volume_attribute_1 | Volume_attribute_2 | top_surface_TWT   | bot_surface_TWT   | Surface_attribute | Fault_polygons_TWT | top_surface_TVDSS | bot_surface_TVDSS | lateral_trend     | model_area_boundary | licence_area_boundary
TWT_cube              | x        | 100.0              | 100.0              | 83.93427877253987 | 83.93427877253987 | 83.93427877253987 | 2.633444831355508  | 83.93427877253987 | 83.93427877253987 | 83.93427877253987 | 84.88020263008978   | 43.16871767287324    
Volume_attribute 1    | 100.0    | x                  | 100.0              | 83.93427877253987 | 83.93427877253987 | 83.93427877253987 | 2.633444831355508  | 83.93427877253987 | 83.93427877253987 | 83.93427877253987 | 84.88020263008978   | 43.16871767287324    
Volume_attribute 2    | 100.0    | 100.0              | x                  | 83.93427877253987 | 83.93427877253987 | 83.93427877253987 | 2.633444831355508  | 83.93427877253987 | 83.93427877253987 | 83.9342787725398

In [288]:
data = table_result

# Создаем DataFrame из данных
df = pd.DataFrame(data[1:], columns=data[0])

# Записываем DataFrame в файл Excel
output_file = "output.xlsx"
df.to_excel(output_file, index=False)

print(f"Данные успешно записаны в файл {output_file}")

Данные успешно записаны в файл output.xlsx
